In [1]:
import pandas as pd
import ast
from gensim.models import Word2Vec

def processar_string_entrada(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return []


In [2]:
#selecione todas as pools completas
def get_all_pools(df):
    p = []
    complete_pools = df[df['pick'].isnull()]['pool']

    # Printando as pools completas
    for _, pool in enumerate(complete_pools):
        p.append(pool)
    return p

In [4]:
df_pools = pd.read_csv('handled_data/df_stops_woe.csv')

In [5]:
def extract_card_names(row, prefix):
    """ Extrai os nomes das cartas com base no prefixo e retorna uma lista de nomes. """
    return [name[len(prefix):] for name, value in row.items() if name.startswith(prefix) and value]

# Criação das novas colunas 'pool' e 'pack'
df_pools['pool'] = df_pools.apply(lambda row: extract_card_names(row, 'pool_'), axis=1)
df_pools['pack'] = df_pools.apply(lambda row: extract_card_names(row, 'pack_card_'), axis=1)

In [31]:
df_pools = df_pools[['pool']]

In [32]:
df_pools

,pool
0,"[Archon of the Wild Rose, Armory Mice, Barrow ..."
1,"[Back for Seconds, Beanstalk Wurm, Bespoke Bat..."
2,"[Aquatic Alchemist, Back for Seconds, Beluna's..."
3,"[Aquatic Alchemist, Archon's Glory, Armory Mic..."
4,"[Bellowing Bruiser, Boundary Lands Ranger, Bra..."
...,...
97828,"[Barrow Naughty, Beluna's Gatekeeper, Charming..."
97829,"[Belligerent of the Ball, Bellowing Bruiser, B..."
97830,"[Armory Mice, Barrow Naughty, Candy Grapple, C..."
97831,"[Bellowing Bruiser, Bespoke Battlegarb, Evolvi..."


In [47]:
decks = df_pools['pool'].tolist()

In [57]:
todas_cartas = [carta for deck in decks for carta in deck]

In [58]:
# Converte a lista concatenada em um conjunto para remover duplicatas
cartas_unicas = set(todas_cartas)

# Conta o número de cartas únicas
total_cartas_unicas = len(cartas_unicas)

In [59]:
total_cartas_unicas

324

In [51]:
# Treine o modelo Word2Vec
model = Word2Vec(sentences=decks, vector_size=50, window=22, min_count=1, workers=4)

# Salve o modelo para uso futuro
model.save("card2vec.model")

In [55]:
from gensim.models import Word2Vec
# Carregue o modelo treinado (se necessário)
model = Word2Vec.load("card2vec.model")

# Obtenha o vetor (embedding) de uma carta específica
vector = model.wv['Archon of the Wild Rose']

# Encontre cartas semelhantes a uma dada carta
similar_cards = model.wv.most_similar("Archon of the Wild Rose", topn=10)

In [56]:
similar_cards

[('A Tale for the Ages', 0.8881415724754333),
 ('Break the Spell', 0.8536645770072937),
 ('Discerning Financier', 0.8198558688163757),
 ('Dutiful Griffin', 0.8170870542526245),
 ('Expel the Interlopers', 0.7901948690414429),
 ('Eerie Interference', 0.7870287895202637),
 ('Dawn of Hope', 0.7736096978187561),
 ('Blind Obedience', 0.7703657150268555),
 ('Cheeky House-Mouse', 0.7637557983398438),
 ("Archon's Glory", 0.7413078546524048)]